In [ ]:
from pytubefix import YouTube

yt = YouTube('https://www.youtube.com/watch?v=OMkEVX23BdM')
stream = yt.streams.get_highest_resolution()
stream.download(output_path='../data/', filename='lion.mp4')
print('Download complete!')

Download complete!


In [22]:
import torch
import torchvision.models as models
from torchvision.models.resnet import ResNet50_Weights
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
imagenet_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from torchvision.datasets import ImageNet

val_dir = '../data/'
imagenet_data = ImageNet(val_dir, split='val', transform=imagenet_transform)
val_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=32, shuffle=False)


AttributeError: 'int' object has no attribute 'size'

In [33]:
class_idx = 291
specific_class_data = [(img, label) for img, label in imagenet_data if label == class_idx]

In [54]:
# Evaluate model on specific class
correct = 0
for inputs, labels in tqdm(specific_class_data[:]):
    inputs = inputs.unsqueeze(0)  # Add batch dimension
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    correct += (preds == labels).sum().item()

print(f"Validation accuracy for class {class_idx}: {correct / 50:.4f}")

100%|██████████| 50/50 [00:01<00:00, 38.39it/s]

Validation accuracy for class 291: 1.0000


In [55]:
import cv2
import os

video_path = '../data/lion_trimed.mp4'
output_dir = '../data/lion_frames'
os.makedirs(output_dir, exist_ok=True)

# Extract frames from the video
cap = cv2.VideoCapture(video_path)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames.")

Extracted 148 frames.


In [ ]:
frame_paths = sorted([os.path.join(output_dir, fname) for fname in os.listdir(output_dir)])
frame_preds = []

for frame_path in frame_paths:
    frame = Image.open(frame_path).convert('RGB')
    input_tensor = imagenet_transform(frame).unsqueeze(0)  # Add batch dimension
    outputs = model(input_tensor)
    _, preds = torch.max(outputs, 1)
    frame_preds.append(preds.item())

correct_frames = sum([1 for pred in frame_preds if pred == class_idx])
print(f"Accuracy on video frames: {correct_frames / len(frame_preds):.4f}")

Accuracy on video frames: 0.9595
